In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import pandas as pd
import rpy2

Call R

In [63]:
# Call R
from rpy2.robjects.packages import importr
from rpy2 import robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()  # make pyr2 accept and auto-convert pandas dataframes
from rpy2.robjects import IntVector, FloatVector, FactorVector, globalenv
lme4 = importr('lme4')
base = importr('base')
stats = importr('stats')
broom = importr('broom')
utils = importr('utils')
ez = importr('ez')

Load our data

In [6]:
dname = os.getcwd()+ '/data_loading/data_v2_quotes/'
dname

'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts/data_loading/data_v2_quotes/'

In [7]:
DF = pd.read_csv(dname + 'full_data.csv')

In [9]:
pd.set_option('display.max_columns', None)
DF.head(3)

,Unnamed: 0,id,expt_index,expt_trial,expt_block,trial_att,vbxi_category,exemplar,trialProperties,trialPropShuff,trial_bonus,block_bonus,resp_timestamp,resp_reactiontime,property_order,resp_correct_array,resp_correct_array_unshuf,prop_responses,groundTruthCorrect,groundTruthCorrect_unshuf,resp_indices,resp_indices_values
0,0,0,0,0,0,1,7,10,"[1, 0, 1, 0, 0, 0, 1]","[[0, 1, 0, 0, 1, 0, 1], [0, 0, 0, 1, 1, 1, 0]]",1,23.0,"[1656927914973, 1656927926827]","[9.699, 21.552]","[[1, 0, 5, 4, 2, 3, 6], [5, 3, 4, 2, 0, 6, 1]]","[[False, True, False, None, None, None, None],...","[[True, False, None, None, None, False, None],...","[[1, 1, 1, None, None, None, None], [None, Non...","[[0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1]]","[[1, 0, 0, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 2, 1], [4, 3, 5]]","[[1, 1, 1], [1, 1, 1]]"
1,1,0,1,1,0,0,7,0,"[1, 0, 1, 0, 0, 0, 1]","[[1, 1, 1, 0, 0, 0, 0]]",3,32.0,[1656927934679],[5.846],"[[0, 6, 2, 3, 1, 5, 4]]","[[True, True, True, None, None, None, None]]","[[True, None, True, None, None, None, True]]","[[1, 1, 1, None, None, None, None]]","[[1, 1, 1, 1, 1, 1, 1]]","[[1, 1, 1, 1, 1, 1, 1]]","[[0, 1, 2]]","[[1, 1, 1]]"
2,2,0,2,2,0,1,3,1,"[1, 1, 0, 0, 1, 0, 0]","[[1, 0, 0, 0, 1, 0, 1], [1, 1, 1, 0, 0, 0, 0]]",1,31.0,"[1656927943664, 1656927953941]","[6.981, 17.258]","[[4, 2, 5, 3, 1, 6, 0], [0, 4, 1, 2, 5, 3, 6]]","[[None, None, False, False, True, None, None],...","[[None, True, None, False, None, False, None],...","[[None, None, 1, 1, 1, None, None], [1, 1, 1, ...","[[0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1]]","[[0, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1]]","[[2, 4, 3], [1, 2, 0]]","[[1, 1, 1], [1, 1, 1]]"


In [11]:
ACCURACY = np.load(dname + 'ACCURACY_New.npy')
acc_first_prop = np.load(dname + 'Acc_first_prop_New.npy')
acc_second_prop = np.load(dname + 'acc_second_prop_New.npy')
acc_third_prop = np.load(dname + 'acc_third_prop_New.npy')

ACCURACY.shape

(49, 8, 16)

In [14]:
# we get the average accuracy for each subject across blocks for each level of the hierarchy
subj_acc_first = np.nanmean(acc_first_prop, axis=2)
subj_acc_second = np.nanmean(acc_second_prop, axis=2)
subj_acc_third = np.nanmean(acc_third_prop, axis=2)
subj_acc_third.shape

(49, 8)

In [42]:
# now lets construct a dataframe with this
ids = np.array(range(subj_acc_first.shape[0]))
block = np.full(subj_acc_first.shape[0], 0)
level = np.full(subj_acc_first.shape[0], 0)
test = np.array([ids, block, level, subj_acc_first[:,0]]).T
dataframe_dict = {'Id' : ids, 'block': block, 'level':level, 'Accuracy': subj_acc_first[:,0]}
df_first = pd.DataFrame(data = dataframe_dict)

test.shape

(49, 4)

In [58]:
# a function to create a long format dataframe to run the ANOVA fml

def construct_long_df(input_list):
    """ a function to create my desired long df"""

    # initalise an empty array
    n_parti = input_list[0].shape[0]
    n_blocks = input_list[0].shape[1]
    long_array = np.full((len(input_list)*n_parti*n_blocks, 4), np.nan)

    # initialise the participant ids
    ids = np.array(range(n_parti))

    # row counter
    current_row = 0

    for level, arr in enumerate(input_list):
        for block in range(n_blocks):

            # get the block, level and accuracy arrays
            blocks = np.full(subj_acc_first.shape[0], block)
            levels = np.full(subj_acc_first.shape[0], level)
            accs   = arr[:,block]
            
            # insert the desired values in our array
            long_array[current_row:current_row + n_parti,:] = np.array([ids, blocks, levels, accs]).T

            # update the row counter
            current_row += 49

    # create the dataframe
    df_long = pd.DataFrame(long_array, columns=['Id', 'block', 'level', 'Accuracy'])
    df_long = df_long.astype({"Id": str,'block': str, 'level': str})

    return df_long

input_list = [subj_acc_first, subj_acc_second, subj_acc_third]
df_long = construct_long_df(input_list=input_list)

df_long 

,Id,block,level,Accuracy
0,0.0,0.0,0.0,0.6250
1,1.0,0.0,0.0,0.6250
2,2.0,0.0,0.0,0.8125
3,3.0,0.0,0.0,0.8125
4,4.0,0.0,0.0,0.4375
...,...,...,...,...
1171,44.0,7.0,2.0,0.3750
1172,45.0,7.0,2.0,0.8125
1173,46.0,7.0,2.0,0.2500
1174,47.0,7.0,2.0,1.0000


In [59]:
globalenv['accuracy'] = FloatVector(df_long['Accuracy'])
globalenv['suj'] = FactorVector(df_long['Id'])
globalenv['block'] = FactorVector(df_long['block'])
globalenv['level'] = FactorVector(df_long['level'])

In [72]:
anova1 = robjects.r("""
library(ez)
function(df) {
    # df gets passed in
    ezANOVA(
        data=df,
        dv=.(Accuracy),
        within = .(block, level),
        wid=Id, 
        detailed=TRUE)
}
""")
print(anova1(df_long))

R[write to console]: Warning:
R[write to console]:  Converting "Id" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "block" to factor for ANOVA.

R[write to console]: Warning:
R[write to console]:  Converting "level" to factor for ANOVA.



$ANOVA
       Effect DFn DFd        SSn       SSd           F            p p<.05
1 (Intercept)   1  48 643.899902 19.522461 1583.160822 2.066082e-38     *
2       block   7 336   7.723849  8.904569   41.635337 4.569080e-42     *
3       level   2  96  14.894133  7.230867   98.870348 4.861178e-24     *
4 block:level  14 672   0.251807  7.435693    1.625502 6.747331e-02      
          ges
1 0.937272200
2 0.151992088
3 0.256849758
4 0.005809313

$`Mauchly's Test for Sphericity`
       Effect         W            p p<.05
2       block 0.1437335 2.256692e-08     *
3       level 0.5532671 9.098486e-07     *
4 block:level 0.0270636 9.412676e-04     *

$`Sphericity Corrections`
       Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF] p[HF]<.05
2       block 0.5891214 8.289681e-26         * 0.6514071 2.819595e-28         *
3       level 0.6912126 2.227233e-17         * 0.7050864 1.117008e-17         *
4 block:level 0.6708404 1.017877e-01           0.8477032 8.140385e-02          
